In [2]:
from textblob import TextBlob, Word, Blobber
from textblob.classifiers import NaiveBayesClassifier
from textblob.taggers import NLTKTagger
from nltk.stem.porter import PorterStemmer

In [3]:
import pandas as pd
import textstat
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.linear_model import Ridge
import datetime as dt
from sklearn.metrics import r2_score
import seaborn as sns
from scipy import stats
from spellchecker import SpellChecker

In [4]:
from collections import OrderedDict
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
path_review="D:/Trinity_DS/Dissertations/201907/politness/politness_predicted/yelp_polite.csv"

In [6]:
df_review = pd.read_csv(path_review,low_memory=False,encoding='iso-8859-1')

In [7]:
df_review=df_review.drop(df_review.columns[0], axis=1)
df_review.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,politness
0,ymAUG8DZfQcFTBSOiaNN4w,u0LXt3Uea_GidxRW1xcsfg,9_CGhHMz8698M9-PkVf0CQ,4,5/11/2012,Who would have guess that you would be able to...,0,0,2,0.067161
1,w41ZS9shepfO3uEyhXEWuQ,u0LXt3Uea_GidxRW1xcsfg,5r6-G9C4YLbC7Ziz57l3rQ,3,2/9/2013,"Not bad!! Love that there is a gluten-free, ve...",1,0,0,0.045530
2,PIsUSmvaUWB00qv5KTF1xA,u0LXt3Uea_GidxRW1xcsfg,z8oIoCT1cXz7gZP5GeU5OA,4,5/1/2013,This is currently my parents new favourite res...,1,0,0,-0.531268
3,PdZ_uFjbbkjtm3SCY_KrZw,u0LXt3Uea_GidxRW1xcsfg,XWTPNfskXoUL-Lf32wSk0Q,3,9/28/2011,Server was a little rude.\r\n\r\nOrdered the c...,5,0,1,-0.053217
4,lsoSqIrrDbQvWpMvsSj2xw,u0LXt3Uea_GidxRW1xcsfg,RtUvSWO_UZ8V3Wpj0n077w,3,12/3/2012,Wanted to check out this place due to all the ...,2,1,1,-0.157010


In [8]:
analysis_df = pd.DataFrame()
df_review['date'] =  pd.to_datetime(df_review['date'])

In [9]:
analysis_df['day_of_week'] = df_review['date'].dt.weekday_name
analysis_df['stars'] = df_review['stars']

In [10]:
analysis_df.stars.unique()

array([4, 3, 5, 1, 2], dtype=int64)

In [11]:
np_matix= np.array(analysis_df)
np_matix
days_rating=pd.crosstab(np_matix[:,0],np_matix[:,1],
            rownames=['Days'], colnames=['Ratings'],)
chi2 , p ,dof ,expected = stats.chi2_contingency(days_rating)
days_rating

Ratings,1,2,3,4,5
Days,,,,,
Friday,1979,2141,3783,6778,5881
Monday,2273,2644,4626,8125,6581
Saturday,2276,2526,4035,7164,6564
Sunday,2678,2739,4698,8233,7046
Thursday,1780,2246,3769,6621,5580
Tuesday,1825,2251,3974,7249,5619
Wednesday,1879,2302,4075,7339,6021


In [12]:
days_rating=pd.crosstab(np_matix[:,0],np_matix[:,1],
            rownames=['Days'], colnames=['Ratings'],)
chi2 , p ,dof ,expected = stats.chi2_contingency(days_rating)
print("Reviews and DaysOfWeek chi_2 value---------------",chi2)
print("Reviews and DaysOfWeek p value-------------------",p)
print("Reviews and DaysOfWeek degreeoffreedom value-----",dof)

Reviews and DaysOfWeek chi_2 value--------------- 161.90234724872974
Reviews and DaysOfWeek p value------------------- 1.9730334914915967e-22
Reviews and DaysOfWeek degreeoffreedom value----- 24


In [13]:
prob=0.95    
critical_day = stats.chi2.ppf(prob, dof)
if abs(chi2) >= critical_day:
    print('Dependent (reject H0)')
    print('Reviews Dependent on Days of the Week')
else:
    print('Independent (fail to reject H0)')
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
    print('Dependent (reject H0)')
else:
    print('Independent (fail to reject H0)')

Dependent (reject H0)
Reviews Dependent on Days of the Week
significance=0.050, p=0.000
Dependent (reject H0)


In [14]:
transformed_df=analysis_df
transformed_df['Weekend_WeekDays'] = np.where((transformed_df['day_of_week']=='Sunday') | (transformed_df['day_of_week']=='Saturday'),
                                              'Weekend','Weekday')
transformed_df['high_low_Rating'] = np.where(transformed_df['stars']<=3,
                                              'low','high')
transformed_df

,day_of_week,stars,Weekend_WeekDays,high_low_Rating
0,Friday,4,Weekday,high
1,Saturday,3,Weekend,low
2,Wednesday,4,Weekday,high
3,Wednesday,3,Weekday,low
4,Monday,3,Weekday,low
5,Friday,5,Weekday,high
6,Wednesday,3,Weekday,low
7,Sunday,3,Weekend,low
8,Tuesday,1,Weekday,low
9,Friday,3,Weekday,low


In [17]:
transformedMatrix= np.array(transformed_df)
crWeekendWeekdays=pd.crosstab(transformedMatrix[:,3],transformedMatrix[:,2],
            rownames=['high_low_Rating'], colnames=['WeekendWeekdays'])
crWeekendWeekdays

WeekendWeekdays,Weekday,Weekend
high_low_Rating,,
high,65794,29007
low,41547,18952


In [16]:

chi2_v4 , p_v4  ,dof_v4  ,expected_v4  = stats.chi2_contingency(crWeekendWeekdays)
print("HighLowRating and WeekendWeekdays chi_2 value---------------",chi2_v4)
print("HighLowRating and WeekendWeekdays p value-------------------",p_v4)
print("HighLowRating and WeekendWeekdays degreeoffreedom value-----",dof_v4)
prob=0.95
critical_hour = stats.chi2.ppf(prob, dof_v4)
if abs(chi2_v4) >= critical_hour:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
    print('High Low Ratings Dependent on Six hours bands')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
if p_v4 <= alpha:
    print('Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')
else:
    print('Independent (fail to reject H0) Hypothesis tested with probability of 95% and alpha of 0.5')


HighLowRating and WeekendWeekdays chi_2 value--------------- 9.144751181089555
HighLowRating and WeekendWeekdays p value------------------- 0.002494325563510335
HighLowRating and WeekendWeekdays degreeoffreedom value----- 1
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5
High Low Ratings Dependent on Six hours bands
Dependent (reject H0) Hypothesis tested with probability of 95% and alpha of 0.5


In [ ]:
list(analysis_df)

In [ ]:
#textstat.flesch_reading_ease(.id)
readablity = []
for text in df_review['text']:
    readablity.append(textstat.flesch_reading_ease((text)))
analysis_df['flesch_reading_ease']=readablity

In [ ]:
temp_df= df_review[['stars','useful']]
df_review.describe()

In [ ]:
smog = []
for text in df_review['text']:
    smog.append(textstat.smog_index(text))

In [ ]:
coleman_liau=[]
for text in df_review['text']:
    coleman_liau.append(textstat.coleman_liau_index(text))


In [ ]:
sentence_count=[]
for text in df_review['text']:
    sentence_count.append(textstat.sentence_count(text))

In [ ]:
gunning_fog=[]
for text in df_review['text']:
    gunning_fog.append(textstat.gunning_fog(text))

In [ ]:
flesch_kincaid_grade=[]
for text in df_review['text']:
    flesch_kincaid_grade.append(textstat.flesch_kincaid_grade(text))

In [ ]:
subjectivity_list=[]
polarity_list=[]
for text in df_review['text']:
    subjectivity_list.append(TextBlob(text).sentiment.subjectivity)
    polarity_list.append(TextBlob(text).sentiment.subjectivity)

In [ ]:
spell = SpellChecker()
spelling_errors=[]

for text in df_review['text']:
    spelling_errors.append(len(spell.unknown(str(text).split(' '))))

In [ ]:
analysis_df['smog']=smog
analysis_df['coleman_liau']=coleman_liau
analysis_df['sentence_count']=sentence_count
analysis_df['gunning_fog']=gunning_fog
analysis_df['flesch_kincaid_grade']=flesch_kincaid_grade
analysis_df['subjectivity']=subjectivity_list
analysis_df['polarity']=polarity_list
analysis_df['spelling_errors']=spelling_errors

In [ ]:
analysis_df=analysis_df.drop(analysis_df.columns[8], axis=1)

In [ ]:
analysis_df['stars']=df_review['stars']
analysis_df['useful']=df_review['useful']
analysis_df['agg']=df_review['useful']+df_review['funny']+df_review['cool']
analysis_df['politness']=df_review['politness']
analysis_df['usefull_bin'] = np.where(analysis_df['useful']==0, '0', '1')

In [ ]:
analysis_df['stars']=df_review['stars']

In [ ]:
analysis_df.dtypes
analysis_df.groupby('usefull_bin').count()
#analysis_df.to_csv("D:/Trinity_DS/Dissertations/201907/yelp/analysis.csv",index=False)
#analysis_df.groupby('usefull_bin').mean()
#analysis_df.groupby('usefull_bin').var()

In [ ]:
X= np.array(analysis_df.drop(['usefull_bin','agg','useful'], axis=1))
#X= np.array(analysis_df['smog'])
Y= np.array(analysis_df['usefull_bin'])
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=303)
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

# Decision Tree

In [ ]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train_scaled, y_train)
Y_train_Pred=clf.predict(X_train_scaled)
accuracy_score(y_train, Y_train_Pred)

In [ ]:
Y_test_Pred=clf.predict(X_test_scaled)
target_names = ['0', '1']
print(classification_report(y_test, Y_test_Pred, target_names=target_names))

# Support Vector Machines

In [ ]:
list(analysis_df)
list(analysis_df)

In [ ]:
list(analysis_df)
X= np.array(analysis_df.drop(['usefull_bin','agg','useful','date','usefull_diff','flesch_kincaid_grade','subjectivity'], axis=1))
#X= np.array(analysis_df['smog'])
Y= np.array(analysis_df['usefull_bin'])
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=303)
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

In [ ]:
clf_SVM = SVC(gamma='auto',kernel='rbf',C=10)
clf_SVM.fit(X_train_scaled, y_train) 

In [ ]:
Y_train_Pred=clf_SVM.predict(X_train_scaled)
accuracy_score(y_train, Y_train_Pred)

In [ ]:
Y_test_SVM_Pred=clf_SVM.predict(X_test_scaled)
Y_test_SVM_Pred
target_names=['0','1']
print(classification_report(y_test, Y_test_SVM_Pred, target_names=target_names))

# Random Forest

In [ ]:
clf_RF = RandomForestClassifier(n_estimators=100,random_state=0)
clf_RF.fit(X_train_scaled, y_train)
Y_train_Pred=clf_RF.predict(X_train_scaled)
Y_test_RF_Pred=clf_RF.predict(X_test_scaled)
print("Training Accuracy",accuracy_score(y_train, Y_train_Pred))
target_names=['0','1']
print(classification_report(y_test, Y_test_RF_Pred, target_names=target_names))

In [ ]:
clf_RF = RandomForestClassifier(n_estimators=1000,random_state=0,max_depth=4)
clf_RF.fit(X_train_scaled, y_train)
Y_train_Pred=clf_RF.predict(X_train_scaled)
Y_test_RF_Pred=clf_RF.predict(X_test_scaled)
print("Training Accuracy",accuracy_score(y_train, Y_train_Pred))
target_names=['0','1']
print(classification_report(y_test, Y_test_RF_Pred, target_names=target_names))

In [ ]:
clf_RF = RandomForestClassifier(n_estimators=2000,random_state=0)
clf_RF.fit(X_train_scaled, y_train)

In [ ]:
Y_train_Pred=clf_RF.predict(X_train_scaled)
accuracy_score(y_train, Y_train_Pred)

In [ ]:
Y_test_RF_Pred=clf_RF.predict(X_test)
target_names=['0','1']
print(classification_report(y_test, Y_test_RF_Pred, target_names=target_names))

# Regression Analysis

In [ ]:
sns.set(style="whitegrid")
ax = sns.boxplot(x=analysis_df["useful"])


In [ ]:
print(list(analysis_df))
analysis_df['log_usefull']=np.log(analysis_df['useful']+1)
print(analysis_df.dtypes)
sns.distplot(analysis_df["log_usefull"],bins=int(180/5),  hist=True, kde=True, color = 'darkblue', 
            )

In [ ]:
analysis_df['date'] =  pd.to_datetime(df_review['date']).dt.date
#filtered_df = data_df[data_df['reviews_dateAdded_Date_time'].notnull()]
#filtered_df = data_df[data_df['reviews_date_Date_time'].notnull()]
#analysis_df['date']-dt.datetime.now().date()-

In [ ]:
analysis_df['diff_days'] = dt.datetime.now().date() - analysis_df['date']
analysis_df['diff_days']=(analysis_df['diff_days']/np.timedelta64(1,'M'))


analysis_df['usefull_diff'] = (analysis_df['useful']/analysis_df['diff_days'])
analysis_df['usefull_diff'].describe()
#usefull_diff

In [ ]:
analysis_df['diff_days'] = dt.datetime.now().date() - analysis_df['date']
analysis_df['diff_days']=(analysis_df['diff_days']/np.timedelta64(1,'M'))
#(analysis_df['diff_days']/np.timedelta64(1,'M')).describe()
corr_df = analysis_df.drop(['agg','log_usefull','usefull_diff','flesch_kincaid_grade','subjectivity'], axis=1)
corr = abs(corr_df.corr())

plt.figure(figsize= (10, 10))
sns.heatmap(corr_df.corr())
fig, ax = plt.subplots(figsize=(10, 10)) 
mask = np.zeros_like(corr_df.corr())

mask[np.triu_indices_from(mask)] = 1
sns.heatmap(corr_df.corr(), mask= mask, ax= ax, annot= True,annot_kws={"size": 11},fmt='.2f')


corr = np.round(abs(corr_df.corr()),2)
corr.style.background_gradient(cmap='coolwarm') 

In [ ]:
analysis_df[['subjectivity','polarity']]

In [ ]:
X= np.array(analysis_df.drop(['usefull_bin','agg','useful','usefull_diff','date','subjectivity'], axis=1))
#X= np.array(analysis_df['smog'])
Y= np.array(analysis_df['usefull_diff'])
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=303)
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

In [ ]:
reg = LinearRegression().fit(X_train_scaled, y_train)
y_pred=reg.predict(X_test_scaled)
#scaler.transform(X_test)
(mean_squared_error(y_test, y_pred)**0.5)*100


In [ ]:

X= np.array(analysis_df.drop(['usefull_bin','agg','useful','usefull_diff','date','log_usefull','subjectivity'], axis=1))
#X= np.array(analysis_df['smog'])
Y= np.array(analysis_df['log_usefull'])
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=303)
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)
reg = LinearRegression().fit(X_train_scaled, y_train)
y_pred=reg.predict(X_test_scaled)
#scaler.transform(X_test)
print(r2_score(y_test, (np.exp(y_pred))))
(mean_squared_error(y_test, (np.exp(y_pred)-1))**0.5)*100


In [ ]:
#analysis_df.to_csv("D:/Trinity_DS/Dissertations/201907/dataset_v2/analysis_df.csv", index = None, header=True)
clf_ridge = Ridge(alpha=100000)
clf_ridge.fit(X_train_scaled, y_train) 
y_pred_0=clf_ridge.predict(X_train_scaled)
#scaler.transform(X_test)
mean_squared_error(y_train,np.exp(y_pred_0)-1)*100

In [ ]:
y_pred=clf_ridge.predict(X_test_scaled)
#scaler.transform(X_test)
mean_squared_error(np.exp(y_pred)-1,y_test)*100


In [ ]:
r2_score(y_test,(np.exp(y_pred)-1))

In [ ]:
print(((np.exp(y_pred))-1).mean())
print(y_test.mean())

In [ ]:
y_pred.std()

In [ ]:
((sum((y_pred-y_test)**2))/len(y_pred))**0.5*100

# Politness Vs Helpfull and Polite vs Stars

In [ ]:
X= np.array(analysis_df['politness'])
#X= np.array(analysis_df['smog'])
Y= np.array(analysis_df['log_usefull'])
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=303)
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)
reg = LinearRegression().fit(X_train_scaled.reshape(-1, 1), y_train)
y_pred=reg.predict(X_test_scaled.reshape(-1, 1))
#scaler.transform(X_test)
print(r2_score(y_test, (np.exp(y_pred))-1))
(mean_squared_error(y_test, (np.exp(y_pred)-1))**0.5)*100

# Merged Analysis

In [ ]:
list(analysis_df)
review_roll_up_df=analysis_df.drop(['usefull_bin','date','diff_days'], axis=1)
review_roll_up_df['user_id'] = df_review['user_id']
#business_id
review_roll_up_df['business_id'] = df_review['business_id']
g1 = review_roll_up_df.groupby(['user_id','business_id']).sum()

In [ ]:
review_roll_up_df=g1.reset_index()
join_review_user=pd.merge(df_user, review_roll_up_df,on='user_id')

In [ ]:
list(join_review_user)
join_review_user=join_review_user.drop([
    'user_id',
 'name',
 'yelping_since',
 'funny',
 'cool',
 'fans',
 'compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos'
], axis=1)
g2 = join_review_user.groupby(['business_id']).sum()
business_lvl_review_user=g2.reset_index()

In [ ]:
join_review_user=pd.merge(df_business.dropna(subset='business_id'),business_lvl_review_user.dropna(subset='business_id') ,on='business_id')

In [ ]:
list(analysis_df)

In [ ]:
join_review_user=pd.merge(df_user, review_roll_up_df1,on='user_id')
list(join_review_user)
join_review_user['business_id']

In [ ]:
merged_final_df=pd.merge(df_business, join_review_user,on='business_id',how='right')

In [ ]:
from spellchecker import SpellChecker

spell = SpellChecker()  # loads default word frequency list


# if I just want to make sure some words are not flagged as misspelled
spell.word_frequency.load_words(['microsoft', 'apple', 'google'])
spell.known(['microsoft', 'google','asd'])  # will return both now!

In [ ]:
misspelled = spell.unknown(['something', 'is', 'hapenning', 'here'])
len(misspelled)